In [1]:
# Independant instructions to scrape wikipedia for artist information
# Search based on artist/band name -- wiki link
import urllib.request
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [233]:
url= "https://en.wikipedia.org/wiki/Khalid_(singer)"

In [239]:
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Khalid (singer) - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"0a945402-3aee-4cf0-abee-4cbd0b308595","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Khalid_(singer)","wgTitle":"Khalid (singer)","wgCurRevisionId":958606279,"wgRevisionId":958606279,"wgArticleId":53109702,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 Danish-language sources (da)","Articles with hCards","Official website different in Wikidata and Wikipedia","Wikipedia articles with GND ide

In [240]:
artist_table=soup.find('table', class_='infobox')
print(artist_table.prettify())

<table class="infobox biography vcard" style="width:22em">
 <tbody>
  <tr>
   <th colspan="2" style="text-align:center;font-size:125%;font-weight:bold">
    <div class="fn" style="display:inline">
     Khalid
    </div>
   </th>
  </tr>
  <tr>
   <td colspan="2" style="text-align:center">
    <a class="image" href="/wiki/File:Khalid-MTV_smiling.jpg">
     <img alt="Khalid-MTV smiling.jpg" data-file-height="831" data-file-width="642" decoding="async" height="285" src="//upload.wikimedia.org/wikipedia/commons/thumb/d/df/Khalid-MTV_smiling.jpg/220px-Khalid-MTV_smiling.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/d/df/Khalid-MTV_smiling.jpg/330px-Khalid-MTV_smiling.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/d/df/Khalid-MTV_smiling.jpg/440px-Khalid-MTV_smiling.jpg 2x" width="220"/>
    </a>
    <div>
     Khalid in October 2017
    </div>
   </td>
  </tr>
  <tr>
   <th scope="row">
    Born
   </th>
   <td>
    <div class="nickname" style="display:inline">
     

In [242]:
table_img=artist_table.find('img')['src']
image_src= 'https://'+ table_img
print(image_src)

https:////upload.wikimedia.org/wikipedia/commons/thumb/d/df/Khalid-MTV_smiling.jpg/220px-Khalid-MTV_smiling.jpg


In [258]:
# Row information
row_titles = []
row_info = []

for row in artist_table.select('tr'):
    table_row = row.find(text=True)
#     print(table_row)
    
    row_titles.append(table_row)
    table_data= row.select('td')
    
#     print(table_data)
    
    for item in table_data:
        text=item.find(text=True)
        row_info.append(text)

# get index of titles then get information from within the rows
print('titles: ',row_titles)
print('---------')
print('data: ',row_info)

titles:  ['Khalid', 'Khalid in October 2017', 'Born', 'Occupation', 'Years\xa0active', 'Musical career', 'Origin', 'Genres', 'Instruments', 'Labels', 'Associated acts', 'Website', None]
---------
data:  ['Khalid in October 2017', 'Khalid Donnel Robinson', '\n', '2016–present', 'Musical career', 'El Paso, Texas', '\n', 'Vocals', '\n', '\n', 'khalidofficial', None]


In [259]:
# This is Band Specific
# Look for string in the list for 'Origin', 'Genres', 'Image' etc
band_dict= {'Origin':'null', 
             'Genres':'null', 
             'Members':'null',
             }

for item in band_dict:
#     print(item)
    if item in row_titles:
        item_index= row_titles.index(item)-1
#         print(f'{item}: index {item_index}')
#         print(item_index)
        
    else:
        print(f'{item} not available')
    
#   update the dictionary
    band_dict.update({item: row_info[int(item_index-1)]})
    
    
band_dict.update({'img':image_src})
band_dict

Members not available


{'Origin': 'Musical career',
 'Genres': 'El Paso, Texas',
 'Members': 'El Paso, Texas',
 'img': 'https:////upload.wikimedia.org/wikipedia/commons/thumb/d/df/Khalid-MTV_smiling.jpg/220px-Khalid-MTV_smiling.jpg'}

In [257]:
artist_dict = {'Origin':'null', 
             'Genres':'null', 
             'Born':'null'
             }
for item in artist_dict:
    
    if item in row_titles:
        item_index= row_titles.index(item)-1
#         print(f'{item}: index {item_index}')
#         print(item_index)
        
    else:
        print(f'{item} not available')
    
#   update the dictionary
    artist_dict.update({item: row_info[int(item_index-1)]})

artist_dict.update({'img':image_src})
artist_dict

{'Origin': ['Musical career'],
 'Genres': ['El Paso, Texas', ', U.S.'],
 'Born': ['Khalid in October 2017'],
 'img': 'https:////upload.wikimedia.org/wikipedia/commons/thumb/d/df/Khalid-MTV_smiling.jpg/220px-Khalid-MTV_smiling.jpg'}

In [ ]:
# ---------------------------------
# Step 1: Get Info Table
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')

info_table = soup.find(class_="infobox")
# print(info_table)
members= info_table.find(text = "Members") 

# ---------------------------------
# Step 2: Artist or band?
if info_table.find(text = "Members"):
    band.append(True)
    ##band has "members section"
else:
    band.append(False)
    ##artist does not have members

# ---------------------------------
# Step 2: Genre 
genre=info_table.select("tbody")[0]
# print(genre)
columns = info_table.findAll('td', text = re.compile('Genres'), attrs = {'class' : 'pos'})
genres=genre.find(text="Genres")
# print(genres)
Tables = pd.read_html(url)
print(Tables[0])
